# Using the Segment-Geospatial Python Package with ArcGIS Pro

## Steps to install segment-geospatial

1. Open Windows Registry Editor (`regedit.exe`) and navigate to `Computer\HKEY_LOCAL_MACHINE\SYSTEM\CurrentControlSet\Control\FileSystem`. Change the value of `LongPathsEnabled` to `1`. See [this screenshot](https://user-images.githubusercontent.com/46331011/225140182-df32dcfe-dca2-4e7f-9992-4c389af36184.png). This is a [known issue](https://github.com/Esri/deep-learning-frameworks/blob/master/README.md#known-issues) with the deep learning libraries for ArcGIS Pro. A future release might fix this issue.
2. Download the [Deep Learning Libraries Installer for ArcGIS Pro 3.1](https://github.com/Esri/deep-learning-frameworks/blob/master/README.md#download). Unzip the file and run `ProDeepLearning.msi` to install the libraries.
3. Open ArcGIS Pro and navigate to **Package Manager**. Clone the `arcgispro-py3` environment and name it `segment` or any other name you prefer. Activate the cloned environment.
4. Navigate to Start menu - ArcGIS - Python Command Prompt - Right Click - More - Run as administrator.
5. Run `conda activate base` to activate the base environment.
6. Run `conda install mamba -n base -c conda-forge` to install mamba in the base environment.
7. Run `conda activate segment` to activate the cloned environment.
8. Run `mamba install -c conda-forge leafmap localtileserver segment-geospatial` to install the required package dependencies in the cloned environment.
9. Download this notebook and run it in ArcGIS Pro.

In [ ]:
import os
import leafmap
from samgeo import SamGeo, tms_to_geotiff
%matplotlib inline

## Create an interactive map

In [ ]:
m = leafmap.Map(center=[37.8713, -122.2580], zoom=17, height="800px")
m.add_basemap("SATELLITE")
m


Pan and zoom the map to select the area of interest. Use the draw tools to draw a polygon or rectangle on the map

In [ ]:
leafmap.arc_zoom_to_bounds(m.bounds)

In [ ]:
if m.user_roi_bounds() is not None:
    bbox = m.user_roi_bounds()
else:
    bbox = [-122.2659, 37.8682, -122.2521, 37.8741]

## Download a sample image

In [ ]:
workspace = os.path.dirname(arcpy.env.workspace)
os.chdir(workspace)

In [ ]:
arcpy.env.overwriteOutput = True

In [ ]:
image = "Image.tif"
tms_to_geotiff(output=image, bbox=bbox, zoom=17, source="Satellite", overwrite=True)

You can also use your own image. Uncomment and run the following cell to use your own image.

In [ ]:
# image = '/path/to/your/own/image.tif'

Display the downloaded image on the map.

In [ ]:
m.layers[-1].visible = False
m.add_raster(image, layer_name="Image")
m

## Initialize SAM class

Specify the file path to the model checkpoint. If it is not specified, the model will to downloaded to the working directory.

In [ ]:
out_dir = os.path.join(os.path.expanduser("~"), "Downloads")
checkpoint = os.path.join(out_dir, "sam_vit_h_4b8939.pth")

In [ ]:
sam = SamGeo(
    model_type="vit_h",
    checkpoint=checkpoint,
    sam_kwargs=None,
)

## Automatic mask generation

Segment the image and save the results to a GeoTIFF file. Set `unique=True` to assign a unique ID to each object. 

In [ ]:
sam.generate(image, output="masks.tif", foreground=True, unique=True)

In [ ]:
sam.show_masks(cmap="binary_r")

Show the object annotations (objects with random color) on the map.

In [ ]:
sam.show_anns(axis="off", alpha=1, output="annotations.tif")

Compare images with a slider.

Add image to the map.

In [ ]:
m.add_raster("annotations.tif", alpha=0.5, layer_name="Masks")
m

Convert the object annotations to vector format, such as GeoPackage, Shapefile, or GeoJSON.

In [ ]:
in_raster = os.path.join(workspace, 'masks.tif')
out_shp = os.path.join(workspace, 'masks.shp')

In [ ]:
arcpy.conversion.RasterToPolygon(in_raster, out_shp)

## Automatic mask generation options

There are several tunable parameters in automatic mask generation that control how densely points are sampled and what the thresholds are for removing low quality or duplicate masks. Additionally, generation can be automatically run on crops of the image to get improved performance on smaller objects, and post-processing can remove stray pixels and holes. Here is an example configuration that samples more masks:

In [ ]:
sam_kwargs = {
    "points_per_side": 32,
    "pred_iou_thresh": 0.86,
    "stability_score_thresh": 0.92,
    "crop_n_layers": 1,
    "crop_n_points_downscale_factor": 2,
    "min_mask_region_area": 100,
}

In [ ]:
sam = SamGeo(
    model_type="vit_h",
    checkpoint=checkpoint,
    sam_kwargs=sam_kwargs,
)

In [ ]:
sam.generate(image, output="masks2.tif", foreground=True)

In [ ]:
sam.show_masks(cmap="binary_r")

In [ ]:
sam.show_anns(axis="off", alpha=0.5, output="annotations2.tif")